<a href="https://colab.research.google.com/github/Lacrxe1/OFI-Project/blob/main/Quant_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
#                      OFI QUANT ASSIGNMENT - ALL-IN-ONE SCRIPT
# ==============================================================================

# Load Data
print("Step 1: Importing libraries and loading data...")
import pandas as pd
import numpy as np

try:
    df = pd.read_csv('first_25000_rows.csv')
    print("   -> Data loaded successfully.")

    # Clean
    print("Step 2: Cleaning data types...")
    cols_to_convert = df.filter(regex=r'(bid|ask)_(px|sz)_\d{2}').columns
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
    print("   -> Data types are now numeric.")

    # Calculate

    def calculate_ofi_for_level(df_input, level_input):
        df_local = df_input.copy()
        level_str = f"{level_input:02d}"

        # Define all column names needed for this level
        bid_px_col, prev_bid_px_col = f'bid_px_{level_str}', f'prev_bid_px_{level_str}'
        bid_sz_col, prev_bid_sz_col = f'bid_sz_{level_str}', f'prev_bid_sz_{level_str}'
        ask_px_col, prev_ask_px_col = f'ask_px_{level_str}', f'prev_ask_px_{level_str}'
        ask_sz_col, prev_ask_sz_col = f'ask_sz_{level_str}', f'prev_ask_sz_{level_str}'

        # Create shifted columns
        df_local[prev_bid_px_col] = df_local[bid_px_col].shift(1)
        df_local[prev_bid_sz_col] = df_local[bid_sz_col].shift(1)
        df_local[prev_ask_px_col] = df_local[ask_px_col].shift(1)
        df_local[prev_ask_sz_col] = df_local[ask_sz_col].shift(1)

        # Calculate bid delta
        bid_conditions = [df_local[bid_px_col] > df_local[prev_bid_px_col], df_local[bid_px_col] == df_local[prev_bid_px_col]]
        bid_outcomes = [df_local[bid_sz_col], df_local[bid_sz_col] - df_local[prev_bid_sz_col]]
        delta_bid = np.select(bid_conditions, bid_outcomes, default=0)

        # Calculate ask delta
        ask_conditions = [df_local[ask_px_col] < df_local[prev_ask_px_col], df_local[ask_px_col] == df_local[prev_ask_px_col]]
        ask_outcomes = [df_local[ask_sz_col], df_local[ask_sz_col] - df_local[prev_ask_sz_col]]
        delta_ask = np.select(ask_conditions, ask_outcomes, default=0)

        return delta_bid - delta_ask
    print("   -> Function 'calculate_ofi_for_level' defined.")

    # CALCULATE ALL OFI FEATURES
    print("Step 4: Calculating all OFI features...")

    # Calculate OFI for each of the 10 levels, Multi-Level
    ofi_level_columns = []
    for i in range(10):
        ofi_col_name = f'ofi_level_{i:02d}'
        df[ofi_col_name] = calculate_ofi_for_level(df, i)
        ofi_level_columns.append(ofi_col_name)

    # Assign Best-Level OFI for clarity
    df['best_level_ofi'] = df['ofi_level_00']

    # Calculate Multi-Level OFI by summing the individual levels
    df['multi_level_ofi'] = df[ofi_level_columns].sum(axis=1)

    # Calculate Integrated OFI using a rolling window
    window_size = 10
    df['integrated_ofi'] = df['multi_level_ofi'].rolling(window=window_size).sum()
    print("   -> Best-Level, Multi-Level, and Integrated OFI calculated.")

    # Final results
    print("Final overview of the first 20 rows:")


    display_cols = ['best_level_ofi', 'multi_level_ofi', 'integrated_ofi']
    print(df[display_cols].head(20))

except Exception as e:
    print("\n--- AN ERROR OCCURRED! ---")
    import traceback
    traceback.print_exc()

In [ ]:
# Cross Asset
lag_steps = 5

df['cross_asset_ofi_proxy'] = df['integrated_ofi'].shift(lag_steps)

print(f"Successfully created the 'cross_asset_ofi_proxy' by lagging the Integrated OFI by {lag_steps} steps.")

print("\n--- ASSIGNMENT COMPLETE: FINAL FEATURE OVERVIEW ---")

final_features = ['best_level_ofi', 'multi_level_ofi', 'integrated_ofi', 'cross_asset_ofi_proxy']

print(df[final_features].head(20))